<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama/blob/main/Exemplos_Chunk_Overlap_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de divisão e sobreposiçao de texto usando Langchain

Cria a divisão de um texto em pedaços(chunks) por tokens e caracteres para submeter a um LLM. Realiza a sobreposição(overlap) de parte do texto para manter a semântica entre os chunks.

Semelhante a uma janela deslizante.

Teste online da chunk(divisão) e overlap(overlap) usando um arquivo texto: https://chunkerizer.streamlit.app/

Entendendo o chunk e overlap
https://gustavo-espindola.medium.com/chunk-division-and-overlap-understanding-the-process-ade7eae1b2bd

LangChain: como dividir corretamente seus chunks(pedaços):
https://www.youtube.com/watch?v=n0uPzvGTFI0

Estratégias de chucking: https://www.pinecone.io/learn/chunking-strategies/

Como agrupar dados de texto - uma análise comparativa: https://towardsdatascience.com/how-to-chunk-text-data-a-comparative-analysis-3858c4a0997a

# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Tratamento de logs

Método para tratamento dos logs.

In [1]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [2]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [3]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

# 1 - Instalação das bibliotecas

Biblioteca para manipular pdf

https://pypi.org/project/pypdf/

In [4]:
!pip install pypdf==3.16.4

Biblioteca que persiste os embeddings e realiza busca semântica.

https://pypi.org/project/chromadb/

In [5]:
!pip install chromadb==0.4.15

Biblioteca para realizar a divisão por token.

In [6]:
!pip install tiktoken==0.5.1

Bibioteca LangChain é um framework de código aberto para o desenvolvimento de aplicações usando modelos de linguagem grandes.

https://pypi.org/project/langchain/

In [7]:
!pip install langchain==0.0.323

# 2 - Tipos de divisão

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

In [8]:
#Import de bibliotecas
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, TokenTextSplitter

chunk_size =26
chunk_overlap = 4

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

t_splitter = TokenTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

## Divisor recursivo de caracteres

In [9]:
# Divisor recursivo do texto
text1 = 'abcdefghijklmnopqrstuvwxyz'
r_splitter.split_text(text1)
# Saída - ['abcdefghijklmnopqrstuvwxyz']

['abcdefghijklmnopqrstuvwxyz']

In [10]:
# Dividor de caracter do texto
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'
r_splitter.split_text(text2)
# Saída - ['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

In [11]:
# Dividor recursivo de texto
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"
r_splitter.split_text(text3)
# Saída - ['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

## Divisor de caracter

In [12]:
# Dividor de caracter texto
c_splitter.split_text(text3)
# Saída - ['a b c d e f g h i j k l m n o p q r s t u v w x y z']

['a b c d e f g h i j k l m n o p q r s t u v w x y z']

In [13]:
# Character Text Splitter with separator defined
# Divisor de caracter texto com separador definido
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator = ' '
)
c_splitter.split_text(text3)
# Saída - ['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

## Divisor de token

In [14]:
# Dividor de token do texto
t_splitter.split_text(text1)
# Saída - ['abcdefghijklmnopqrstuvwxyz']

['abcdefghijklmnopqrstuvwxyz']

In [15]:
# Dividor de token do texto
text4 = 'ab cd ef gh ij kl mn op qr st uv wx yz ab cd ef gh ij kl mn op qr st uv wx yz'
t_splitter.split_text(text4)
# Saída - ['ab cd ef gh ij kl mn op qr st uv wx yz ab cd ef gh',' cd ef gh ij kl mn op qr st uv wx yz']

['ab cd ef gh ij kl mn op qr st uv wx yz ab cd ef gh',
 ' cd ef gh ij kl mn op qr st uv wx yz']

# 3 - Divisão por Token

Que palavras ao meio.

## 3.1 - Manipulando documentos Texto

### Cria o texto

In [16]:
# trecho de https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', #As Vítimas algozes


texto = """
No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase
sempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,
é a inimiga hipócrita que rende vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consigna­tários ou fregueses.
Essa parasita das fazendas e estabelecimentos agrícolas
das vizinhanças facilmente se pode conhecer por suas feições e modos
característicos, se nos é lícito dizer assim: uma se parece com todas e não há
hipótese em que alguma delas, por mais dissimulada que seja, chegue a perder o
cará­ter da família.
É uma pequena casa de taipa e coberta de telha, tendo às
vezes na frente varanda aberta pelos três lados, também coberta de telha e com
o teto sustido por esteios fortes, mas rudes e ainda mesmo tortos; as paredes
nem sempre são caiadas, o chão não tem assoalho nem ladrilho; quando há
varanda, abrem-se para ela uma porta e uma janela; dentro está a ven­da: entre
a porta e a janela encostado à parede um banco de pau, defronte um balcão tosco
e no bojo ou no espaço que se vê além, grotesca armação de tábuas contendo
garrafas, botijas, latas de tabaco em pó, a um canto algumas voltas de fumo em
rolo e uma ruim manta de carne-seca. Eis a venda.
Há muitas que nem chegam à opulência da que aí fica
descrita; em todas porém aparece humilde no fundo do quase vazio bojo a porta
baixa que comunica pelo corredor imundo com dois ou mais quartos escuros, onde
se recolhem as pingues colheitas agrícolas do vendelhão que aliás não tem
lavoura.
"""

### Divide e sobrepõe o texto

In [17]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import TokenTextSplitter

# Parâmetros
chunk_tamanho = 250
chunk_sobreposicao = 50

text_splitter = TokenTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    # length_function = len, # Usa o comprimento do texto em caracteres como medida de tamanho
    length_function = lambda x: len(x.split(" ")), # Usa a quantidade de tokens separados por espaço em branco como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.create_documents([texto])

tempo_final = time.time()

print(f"Carregando e dividindo {len(texto)} documentos texto em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 1918 documentos texto em 0.0024573802947998047 segundos!


In [18]:
print(len(chunks))

4


In [19]:
print(chunks)

[Document(page_content='\nNo interior e principalmente longe da vila, ou da freguesia e dos povoados há quase\nsempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,\né a inimiga hipócrita que rende vassalagem à sua vítima.\nA venda de que falo é uma taberna especialíssima\nque não poderia existir, manter-se, medrar em outras condições locais, e em\noutras condições do trabalho rural, e nem se confunde com a taberna regular que\nem toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,\nonde os lavradores ricos e pobres se provêem do que precisa a casa, quando não\nlhes é possível esperar pelas remessas dos seus consign', metadata={'start_index': 0}), Document(page_content='io,\nonde os lavradores ricos e pobres se provêem do que precisa a casa, quando não\nlhes é possível esperar pelas remessas dos seus consigna\xadtários ou fregueses.\nEssa parasita das fazendas e estabelecimentos agrícolas\ndas vizinhanças facilmente se pode co

### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [20]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  qtde char : 650  qtde token : 108  start_index: 0


No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase
sempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,
é a inimiga hipócrita que rende vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consign
-----------------------------------------------------------------------
chunk # 1  qtde char : 612  qtde token : 96  start_index: 512

io,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consigna­tários ou fr

Estatísticas dos chunks

In [21]:
maior_chunk_token = 0
maior_chunk_character = 0
for i, chunk in enumerate(chunks):
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens) )
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  qtde char : 650  qtde token : 108
chunk # 1  qtde char : 612  qtde token : 96
chunk # 2  qtde char : 609  qtde token : 107
chunk # 3  qtde char : 423  qtde token : 70
Maior chunk token: 108
Maior chunk character: 650


## 3.2 - Manipulando Documentos PDF

Gera um documento para cada a página do PDF.

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

### - Link dos PDFs

In [22]:
# As Vítimas Algozes
url1 = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977'

# A Escrava Isaura
url2 = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=92390'

### Download dos pdfs

In [23]:
# Import das bibliotecas
import subprocess

# As Vítimas Algozes

# Arquivo de destino1
destino1 = 'arquivo1.pdf'

# Executa o comando wget no prompt
subprocess.call(["wget", url1, "-O", destino1])

# Arquivo de destino1
destino2 = 'arquivo2.pdf'

# Executa o comando wget no prompt
subprocess.call(["wget", url1, "-O", destino2])

0

### Carrega os pdfs

In [24]:
# Import das bibliotecas
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# Define o diretório
diretorio = '/content'

 # Cria o carregador do PDF
carregador = PyPDFDirectoryLoader(diretorio)

# Carrega os documentos
documentos = carregador.load()

### Divide e sobrepõe os pdfs

In [25]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import TokenTextSplitter

# Parâmetros
chunk_tamanho = 250
chunk_sobreposicao = 50

text_splitter = TokenTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    # length_function = len, # Usa o comprimento do texto em caracteres como medida de tamanho
    length_function = lambda x: len(x.split(" ")), # Usa a quantidade de tokens separados por espaço em branco como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(texto)} documentos pdfs em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 1918 documentos pdfs em 0.29569292068481445 segundos!


### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [26]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  qtde char : 376  qtde token : 47  start_index: 0

J MAS-ALGOZE 
ROMAM 
~% 
LIVRARIA DE B. L. GARNIER 
«9, rua do Ouvidor, CO 
Grandesortimento del.ivros clássicos, Medicina, 
Sciencias e Arlís, Junsprudcncia, Littsratura, 
Novellas, lllustrações, Educação, Devoção, Atlas, 
Happas geographicos, etc, etc. 
Livros francezes, portuguezes, inqlezes, italianos, ele-, 
Encarrega-se áe qualquer conmissio de Liuros-
BIO DE JIMIIIO 
-----------------------------------------------------------------------
chunk # 1  qtde char : 79  qtde token : 14  start_index: 0

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
José Mindlin 
-----------------------------------------------------------------------
chunk # 2  qtde char : 20  qtde token : 3  start_index: 0

AS VICTIMAS-ALGOZES 
-----------------------------------------------------------------------
chunk # 3  qtde char : 155  qtde token : 26  start_index: 0

AS VICTIMAS-ALGOZES 
QUADROS DA ESCRAVIDÃO 
ROMANCES 
POR 
JOAQUI

Estatísticas dos chunks

In [27]:
maior_chunk_token = 0
maior_chunk_character = 0
for i, chunk in enumerate(chunks):
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens) )
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  qtde char : 376  qtde token : 47
chunk # 1  qtde char : 79  qtde token : 14
chunk # 2  qtde char : 20  qtde token : 3
chunk # 3  qtde char : 155  qtde token : 26
chunk # 4  qtde char : 583  qtde token : 93
chunk # 5  qtde char : 119  qtde token : 20
chunk # 6  qtde char : 601  qtde token : 100
chunk # 7  qtde char : 396  qtde token : 61
chunk # 8  qtde char : 625  qtde token : 97
chunk # 9  qtde char : 543  qtde token : 85
chunk # 10  qtde char : 65  qtde token : 9
chunk # 11  qtde char : 619  qtde token : 99
chunk # 12  qtde char : 541  qtde token : 86
chunk # 13  qtde char : 47  qtde token : 7
chunk # 14  qtde char : 583  qtde token : 102
chunk # 15  qtde char : 538  qtde token : 85
chunk # 16  qtde char : 42  qtde token : 8
chunk # 17  qtde char : 598  qtde token : 101
chunk # 18  qtde char : 347  qtde token : 59
chunk # 19  qtde char : 651  qtde token : 97
chunk # 20  qtde char : 355  qtde token : 56
chunk # 21  qtde char : 602  qtde token : 102
chunk # 22  qtde char : 4

## 3.3 - Manipulando Documentos HTML

Gera um único documento para a página HTML.

https://python.langchain.com/docs/integrations/document_loaders/web_base

### Links dos HTML

In [28]:
# As Vítimas Algozes
url1 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979"

# A Escrava Isaura
url2 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=92389"

### Carrega os HTMLs

Se ocorrer o **erro** abaixo, instale o Chromedb. Alguma dependência do pacote resolve o problema.

---------------------------------------------------------------------------
IncompleteRead                            Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/urllib3/response.py in _error_catcher(self)
    709             try:
--> 710                 yield
    711

15 frames
IncompleteRead: IncompleteRead(821958 bytes read, 2240625 more expected)

The above exception was the direct cause of the following exception:

ProtocolError                             Traceback (most recent call last)
ProtocolError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

During handling of the above exception, another exception occurred:

ChunkedEncodingError                      Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/requests/models.py in generate()
    816                     yield from self.raw.stream(chunk_size, decode_content=True)
    817                 except ProtocolError as e:
--> 818                     raise ChunkedEncodingError(e)
    819                 except DecodeError as e:
    820                     raise ContentDecodingError(e)

ChunkedEncodingError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

In [29]:
# Import das bibliotecas
from langchain.document_loaders import WebBaseLoader

# Cria o carregador da página
carregador = WebBaseLoader([url1, url2])
# carregador.requests_kwargs = {'verify':False}

# Carrega os documentos
documentos = carregador.load()

### Divide e sobrepõe os HTMLs

In [30]:
texto = "bom dia texte ."

print(len(texto.split(" ")))

4


In [31]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import TokenTextSplitter

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

text_splitter = TokenTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    # length_function = len, # Usa o comprimento do texto em caracteres como medida de tamanho
    length_function = lambda x: len(x.split(" ")), # Usa a quantidade de tokens separados por espaço em branco como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(texto)} documentos htmls em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 15 documentos htmls em 0.5958163738250732 segundos!


### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [32]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  qtde char : 1199  qtde token : 165  start_index: 0

















As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo

Edição de base:
Biblioteca Nacional – setor de obras digitalizadas
ÍNDICE
SIMEÃO, O CRIOULO
PAI- RAIOL, O FEITICEIRO
LUCINDA, A
MUCAMA
CONCLUSÃO
I
SIMEÃO, O CRIOULO
I
No interior e principalmente longe da vila, ou da
freguesia e dos povoados há quase sempre uma venda perto da fazenda: é a
parasita que se apega à árvore; pior que isso, é a inimiga hipócrita que rende
vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores r

Estatísticas dos chunks

In [33]:
maior_chunk_token = 0
maior_chunk_character = 0
for i, chunk in enumerate(chunks):
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens) )
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  qtde char : 1199  qtde token : 165
chunk # 1  qtde char : 1214  qtde token : 203
chunk # 2  qtde char : 1227  qtde token : 201
chunk # 3  qtde char : 1250  qtde token : 196
chunk # 4  qtde char : 1246  qtde token : 186
chunk # 5  qtde char : 1251  qtde token : 198
chunk # 6  qtde char : 1187  qtde token : 187
chunk # 7  qtde char : 1271  qtde token : 198
chunk # 8  qtde char : 1220  qtde token : 186
chunk # 9  qtde char : 1192  qtde token : 187
chunk # 10  qtde char : 1275  qtde token : 198
chunk # 11  qtde char : 1263  qtde token : 197
chunk # 12  qtde char : 1219  qtde token : 185
chunk # 13  qtde char : 1264  qtde token : 185
chunk # 14  qtde char : 1237  qtde token : 184
chunk # 15  qtde char : 1226  qtde token : 187
chunk # 16  qtde char : 1255  qtde token : 184
chunk # 17  qtde char : 1253  qtde token : 192
chunk # 18  qtde char : 1294  qtde token : 205
chunk # 19  qtde char : 1253  qtde token : 189
chunk # 20  qtde char : 1234  qtde token : 188
chunk # 21  qtde char :

# 4 - Divisão Recursiva de Caracter

## 4.1 - Manipulando documentos Texto

### Cria o texto

In [34]:
# trecho de https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', #As Vítimas algozes


texto = """
No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase
sempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,
é a inimiga hipócrita que rende vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consigna­tários ou fregueses.
Essa parasita das fazendas e estabelecimentos agrícolas
das vizinhanças facilmente se pode conhecer por suas feições e modos
característicos, se nos é lícito dizer assim: uma se parece com todas e não há
hipótese em que alguma delas, por mais dissimulada que seja, chegue a perder o
cará­ter da família.
É uma pequena casa de taipa e coberta de telha, tendo às
vezes na frente varanda aberta pelos três lados, também coberta de telha e com
o teto sustido por esteios fortes, mas rudes e ainda mesmo tortos; as paredes
nem sempre são caiadas, o chão não tem assoalho nem ladrilho; quando há
varanda, abrem-se para ela uma porta e uma janela; dentro está a ven­da: entre
a porta e a janela encostado à parede um banco de pau, defronte um balcão tosco
e no bojo ou no espaço que se vê além, grotesca armação de tábuas contendo
garrafas, botijas, latas de tabaco em pó, a um canto algumas voltas de fumo em
rolo e uma ruim manta de carne-seca. Eis a venda.
Há muitas que nem chegam à opulência da que aí fica
descrita; em todas porém aparece humilde no fundo do quase vazio bojo a porta
baixa que comunica pelo corredor imundo com dois ou mais quartos escuros, onde
se recolhem as pingues colheitas agrícolas do vendelhão que aliás não tem
lavoura.
"""

### Divide e sobrepõe o texto

In [35]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Parâmetros
chunk_tamanho = 250
chunk_sobreposicao = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    # length_function = len, # Usa o comprimento do texto em caracteres como medida de tamanho
    length_function = lambda x: len(x.split(" ")), # Usa a quantidade de tokens separados por espaço em branco como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.create_documents([texto])

tempo_final = time.time()

print(f"Carregando e dividindo {len(texto)} documentos texto em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 1918 documentos texto em 0.0007486343383789062 segundos!


In [36]:
print(len(chunks))

2


In [37]:
print(chunks)

[Document(page_content='No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase\nsempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,\né a inimiga hipócrita que rende vassalagem à sua vítima.\nA venda de que falo é uma taberna especialíssima\nque não poderia existir, manter-se, medrar em outras condições locais, e em\noutras condições do trabalho rural, e nem se confunde com a taberna regular que\nem toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,\nonde os lavradores ricos e pobres se provêem do que precisa a casa, quando não\nlhes é possível esperar pelas remessas dos seus consigna\xadtários ou fregueses.\nEssa parasita das fazendas e estabelecimentos agrícolas\ndas vizinhanças facilmente se pode conhecer por suas feições e modos\ncaracterísticos, se nos é lícito dizer assim: uma se parece com todas e não há\nhipótese em que alguma delas, por mais dissimulada que seja, chegue a perder o\ncará\

### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [38]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  qtde char : 1261  qtde token : 204  start_index: 1

No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase
sempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,
é a inimiga hipócrita que rende vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consigna­tários ou fregueses.
Essa parasita das fazendas e estabelecimentos agrícolas
das vizinhanças facilmente se pode conhecer por suas feições e modos
característicos, se nos é lícito dizer assim: uma se parece com todas e não há
hipótese em que alguma delas, por mais dissimulada que seja, 

Estatísticas dos chunks

In [39]:
maior_chunk_token = 0
maior_chunk_character = 0
for i, chunk in enumerate(chunks):
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens) )
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  qtde char : 1261  qtde token : 204
chunk # 1  qtde char : 883  qtde token : 150
Maior chunk token: 204
Maior chunk character: 1261


## 4.2 - Manipulando Documentos PDF

Gera um documento para cada a página do PDF.

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

### - Link dos PDFs

In [40]:
# As Vítimas Algozes
url1 = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977'

# A Escrava Isaura
url2 = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=92390'

### Download dos pdfs

In [41]:
# Import das bibliotecas
import subprocess

# As Vítimas Algozes

# Arquivo de destino1
destino1 = 'arquivo1.pdf'

# Executa o comando wget no prompt
subprocess.call(["wget", url1, "-O", destino1])

# Arquivo de destino1
destino2 = 'arquivo2.pdf'

# Executa o comando wget no prompt
subprocess.call(["wget", url1, "-O", destino2])

0

### Carrega os pdfs

In [42]:
# Import das bibliotecas
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# Define o diretório
diretorio = '/content'

 # Cria o carregador do PDF
carregador = PyPDFDirectoryLoader(diretorio)

# Carrega os documentos
documentos = carregador.load()

### Divide e sobrepõe os pdfs

In [43]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    # length_function = len, # Usa o comprimento do texto em caracteres como medida de tamanho
    length_function = lambda x: len(x.split(" ")), # Usa a quantidade de tokens separados por espaço em branco como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(texto)} documentos pdfs em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 1918 documentos pdfs em 0.06918859481811523 segundos!


### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [44]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  qtde char : 375  qtde token : 46  start_index: 0

J MAS-ALGOZE 
ROMAM 
~% 
LIVRARIA DE B. L. GARNIER 
«9, rua do Ouvidor, CO 
Grandesortimento del.ivros clássicos, Medicina, 
Sciencias e Arlís, Junsprudcncia, Littsratura, 
Novellas, lllustrações, Educação, Devoção, Atlas, 
Happas geographicos, etc, etc. 
Livros francezes, portuguezes, inqlezes, italianos, ele-, 
Encarrega-se áe qualquer conmissio de Liuros-
BIO DE JIMIIIO
-----------------------------------------------------------------------
chunk # 1  qtde char : 78  qtde token : 13  start_index: 0

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
José Mindlin
-----------------------------------------------------------------------
chunk # 2  qtde char : 19  qtde token : 2  start_index: 0

AS VICTIMAS-ALGOZES
-----------------------------------------------------------------------
chunk # 3  qtde char : 154  qtde token : 25  start_index: 0

AS VICTIMAS-ALGOZES 
QUADROS DA ESCRAVIDÃO 
ROMANCES 
POR 
JOAQUIM M

Estatísticas dos chunks

In [45]:
maior_chunk_token = 0
maior_chunk_character = 0
for i, chunk in enumerate(chunks):
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens) )
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  qtde char : 375  qtde token : 46
chunk # 1  qtde char : 78  qtde token : 13
chunk # 2  qtde char : 19  qtde token : 2
chunk # 3  qtde char : 154  qtde token : 25
chunk # 4  qtde char : 589  qtde token : 94
chunk # 5  qtde char : 875  qtde token : 139
chunk # 6  qtde char : 1047  qtde token : 163
chunk # 7  qtde char : 1039  qtde token : 162
chunk # 8  qtde char : 997  qtde token : 166
chunk # 9  qtde char : 834  qtde token : 136
chunk # 10  qtde char : 873  qtde token : 132
chunk # 11  qtde char : 956  qtde token : 161
chunk # 12  qtde char : 836  qtde token : 127
chunk # 13  qtde char : 1059  qtde token : 166
chunk # 14  qtde char : 916  qtde token : 143
chunk # 15  qtde char : 17  qtde token : 3
chunk # 16  qtde char : 690  qtde token : 118
chunk # 17  qtde char : 1168  qtde token : 209
chunk # 18  qtde char : 1162  qtde token : 198
chunk # 19  qtde char : 1201  qtde token : 192
chunk # 20  qtde char : 1186  qtde token : 197
chunk # 21  qtde char : 1126  qtde token : 184
c

## 4.3 - Manipulando Documentos HTML

Gera um único documento para a página HTML.

https://python.langchain.com/docs/integrations/document_loaders/web_base

### Links dos HTML

In [46]:
# As Vítimas Algozes
url1 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979"

# A Escrava Isaura
url2 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=92389"

### Carrega os HTMLs

Se ocorrer o **erro** abaixo, instale o Chromedb. Alguma dependência do pacote resolve o problema.

---------------------------------------------------------------------------
IncompleteRead                            Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/urllib3/response.py in _error_catcher(self)
    709             try:
--> 710                 yield
    711

15 frames
IncompleteRead: IncompleteRead(821958 bytes read, 2240625 more expected)

The above exception was the direct cause of the following exception:

ProtocolError                             Traceback (most recent call last)
ProtocolError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

During handling of the above exception, another exception occurred:

ChunkedEncodingError                      Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/requests/models.py in generate()
    816                     yield from self.raw.stream(chunk_size, decode_content=True)
    817                 except ProtocolError as e:
--> 818                     raise ChunkedEncodingError(e)
    819                 except DecodeError as e:
    820                     raise ContentDecodingError(e)

ChunkedEncodingError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

In [47]:
# Import das bibliotecas
from langchain.document_loaders import WebBaseLoader

# Cria o carregador da página
carregador = WebBaseLoader([url1, url2])
# carregador.requests_kwargs = {'verify':False}

# Carrega os documentos
documentos = carregador.load()

### Divide e sobrepõe os HTMLs

In [48]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    # length_function = len, # Usa o comprimento do texto em caracteres como medida de tamanho
    length_function = lambda x: len(x.split(" ")), # Usa a quantidade de tokens separados por espaço em branco como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(texto)} documentos htmls em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 1918 documentos htmls em 0.10783028602600098 segundos!


### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [49]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  qtde char : 217  qtde token : 25  start_index: 16

As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo
-----------------------------------------------------------------------
chunk # 1  qtde char : 2516  qtde token : 401  start_index: 235

Edição de base:
Biblioteca Nacional – setor de obras digitalizadas
ÍNDICE
SIMEÃO, O CRIOULO
PAI- RAIOL, O FEITICEIRO
LUCINDA, A
MUCAMA
CONCLUSÃO
I
SIMEÃO, O CRIOULO
I
No interior e principalmente longe da vila, ou da
freguesia e dos povoados há quase sempre uma venda perto da fazenda: é a
parasita que se apega à árvore; pior que isso, é a inimiga hipócrita que rende
vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a tabe

Estatísticas dos chunks

In [50]:
maior_chunk_token = 0
maior_chunk_character = 0
for i, chunk in enumerate(chunks):
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens) )
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  qtde char : 217  qtde token : 25
chunk # 1  qtde char : 2516  qtde token : 401
chunk # 2  qtde char : 2739  qtde token : 419
chunk # 3  qtde char : 2615  qtde token : 404
chunk # 4  qtde char : 2626  qtde token : 406
chunk # 5  qtde char : 2620  qtde token : 403
chunk # 6  qtde char : 2637  qtde token : 407
chunk # 7  qtde char : 2771  qtde token : 415
chunk # 8  qtde char : 2721  qtde token : 410
chunk # 9  qtde char : 2698  qtde token : 417
chunk # 10  qtde char : 2751  qtde token : 406
chunk # 11  qtde char : 2818  qtde token : 401
chunk # 12  qtde char : 2652  qtde token : 399
chunk # 13  qtde char : 2595  qtde token : 405
chunk # 14  qtde char : 2598  qtde token : 400
chunk # 15  qtde char : 2468  qtde token : 391
chunk # 16  qtde char : 2743  qtde token : 404
chunk # 17  qtde char : 2765  qtde token : 402
chunk # 18  qtde char : 2760  qtde token : 420
chunk # 19  qtde char : 2549  qtde token : 393
chunk # 20  qtde char : 2467  qtde token : 392
chunk # 21  qtde char : 2